# MD Simulation - Luteolin + PDE5A (GPU Production)

**Duration:** 50 ns with checkpoint every 10 ns

**Method:** GROMACS (GPU) + ACPYPE (validated workflow)

**Accelerator:** GPU P100/T4 **REQUIRED**

**Note:** This version compiles GROMACS with CUDA for GPU acceleration.

---

In [ ]:
# Configuration
CONFIG = {
    'complex_name': 'Luteolin_PDE5A',
    'total_time_ns': 50,
    'checkpoint_interval_ns': 10,
    'temperature_k': 310,
    'timestep_fs': 2,
    'nvt_time_ps': 100,
    'npt_time_ps': 100,
}

NSTEPS_PER_SEGMENT = int(CONFIG['checkpoint_interval_ns'] * 1e6 / CONFIG['timestep_fs'])
NUM_SEGMENTS = CONFIG['total_time_ns'] // CONFIG['checkpoint_interval_ns']
dt = CONFIG['timestep_fs'] / 1000

print(f"Complex: {CONFIG['complex_name']}")
print(f"Total time: {CONFIG['total_time_ns']} ns ({NUM_SEGMENTS} segments)")
print(f"Steps per segment: {NSTEPS_PER_SEGMENT:,}")

## Step 1: Install GROMACS with GPU (CUDA) Support

Compiles GROMACS from source with CUDA. Takes ~15 min but enables GPU acceleration (~10-20x faster).

In [ ]:
%%bash
set -e
echo '=== Checking GPU ==='
nvidia-smi | head -10
echo ''
echo '=== Installing Build Dependencies ==='
apt-get update -qq
apt-get install -qq -y cmake build-essential libfftw3-dev
echo 'Dependencies: OK'

In [ ]:
%%bash
set -e
cd /kaggle/working
echo '=== Downloading GROMACS 2024.4 ==='
if [ ! -f gromacs-2024.4.tar.gz ]; then
    wget -q https://ftp.gromacs.org/gromacs/gromacs-2024.4.tar.gz
fi
tar -xzf gromacs-2024.4.tar.gz
echo 'Download: OK'

In [ ]:
%%bash
set -e
cd /kaggle/working/gromacs-2024.4
mkdir -p build && cd build
CUDA_PATH=$(ls -d /usr/local/cuda* 2>/dev/null | head -1)
echo "=== Configuring with CUDA at: $CUDA_PATH ==="
cmake .. \
    -DGMX_GPU=CUDA \
    -DCUDA_TOOLKIT_ROOT_DIR=$CUDA_PATH \
    -DGMX_BUILD_OWN_FFTW=OFF \
    -DGMX_DOUBLE=OFF \
    -DCMAKE_INSTALL_PREFIX=/usr/local/gromacs \
    2>&1 | tail -20
echo 'Configure: OK'

In [ ]:
%%bash
set -e
echo '=== Building GROMACS (takes ~10-15 min) ==='
cd /kaggle/working/gromacs-2024.4/build
make -j4 2>&1 | tail -30
echo ''
echo '=== Installing ==='
make install 2>&1 | tail -10
echo ''
echo '=== Verifying GPU GROMACS ==='
source /usr/local/gromacs/bin/GMXRC
gmx --version | head -15
echo ''
echo 'GROMACS GPU: OK'

## Step 2: Install Micromamba + AmberTools + ACPYPE

In [ ]:
%%bash
set -e
cd /kaggle/working
curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba
./bin/micromamba --version
echo 'Micromamba: OK'

In [ ]:
%%bash
set -e
cd /kaggle/working
export MAMBA_ROOT_PREFIX=/kaggle/working/mamba
./bin/micromamba create -n amber -c conda-forge ambertools=23 openbabel acpype python=3.10 -y
echo 'AmberTools + ACPYPE: OK'

In [ ]:
%%bash
set -e
cd /kaggle/working
export MAMBA_ROOT_PREFIX=/kaggle/working/mamba
eval "$(./bin/micromamba shell hook -s bash)"
micromamba activate amber
echo "antechamber: $(which antechamber)"
echo "acpype: $(which acpype)"
echo 'All tools verified!'

## Step 3: Setup Working Directory

In [ ]:
import os
import shutil
from pathlib import Path

WORK_DIR = Path(f"/kaggle/working/{CONFIG['complex_name']}")
TOPOL_DIR = WORK_DIR / 'topol'
OUTPUT_DIR = Path('/kaggle/working/output')

for d in ['input', 'topol', 'em', 'nvt', 'npt', 'md', 'analysis', 'checkpoints']:
    (WORK_DIR / d).mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(exist_ok=True)

print(f'Working directory: {WORK_DIR}')

In [ ]:
# Copy input files
DATASET_DIR = Path('/kaggle/input/luteolin-pde5a-input')

if DATASET_DIR.exists():
    for f in DATASET_DIR.glob('*.pdb'):
        shutil.copy(f, WORK_DIR / 'input')
        print(f'Copied: {f.name}')
else:
    raise FileNotFoundError(f'Dataset not found: {DATASET_DIR}')

## Step 4: Protein Topology (pdb2gmx)

In [ ]:
%%bash
set -e
source /usr/local/gromacs/bin/GMXRC
cd /kaggle/working/Luteolin_PDE5A/topol
printf '1\n' | gmx pdb2gmx -f ../input/PDE5A_1TBF.pdb -o protein.gro -p topol.top -i posre.itp -ff amber99sb-ildn -water tip3p -ignh
echo 'Protein topology: OK'

## Step 5: Ligand Topology (ACPYPE with GAFF2)

ACPYPE generates proper GROMACS `.itp` files with correct atomtypes and moleculetype sections.

In [ ]:
%%bash
set -e
cd /kaggle/working
export MAMBA_ROOT_PREFIX=/kaggle/working/mamba
eval "$(./bin/micromamba shell hook -s bash)"
micromamba activate amber

cd /kaggle/working/Luteolin_PDE5A/topol

# Run ACPYPE with GAFF2 force field
acpype -i ../input/Luteolin_docked.pdb -n 0 -a gaff2 -o gmx

echo '\n=== ACPYPE OUTPUT FILES ==='
ls -la *.acpype/
echo 'ACPYPE: OK'

In [ ]:
%%bash
set -e
cd /kaggle/working/Luteolin_PDE5A/topol

# Find ACPYPE output directory
ACPYPE_DIR=$(ls -d *.acpype 2>/dev/null | head -1)
if [ -z "$ACPYPE_DIR" ]; then
    echo 'ERROR: ACPYPE output directory not found!'
    exit 1
fi

echo "ACPYPE dir: $ACPYPE_DIR"

# Copy GROMACS files from ACPYPE output
cp $ACPYPE_DIR/*_GMX.gro ligand.gro
cp $ACPYPE_DIR/*_GMX.itp ligand.itp

echo '\n=== ligand.itp first 20 lines ==='
head -20 ligand.itp

echo '\nLigand files copied: OK'

## Step 6: Combine Protein + Ligand

In [ ]:
# Combine GRO files
os.chdir(TOPOL_DIR)

with open('protein.gro', 'r') as f:
    protein_lines = f.readlines()
with open('ligand.gro', 'r') as f:
    ligand_lines = f.readlines()

protein_atoms = protein_lines[2:-1]
ligand_atoms = ligand_lines[2:-1]
box = protein_lines[-1]
total_atoms = len(protein_atoms) + len(ligand_atoms)

with open('complex.gro', 'w') as f:
    f.write(f"{CONFIG['complex_name']} complex\n")
    f.write(f' {total_atoms}\n')
    f.writelines(protein_atoms)
    f.writelines(ligand_atoms)
    f.write(box)

print(f'Complex: {total_atoms} atoms')

In [ ]:
# Update topology file
import re

with open('topol.top', 'r') as f:
    topol = f.read()

# Find forcefield include line
ff_pattern = '#include "amber99sb-ildn.ff/forcefield.itp"'
ff_pos = topol.find(ff_pattern)
if ff_pos == -1:
    raise ValueError('Cannot find forcefield include!')

ff_end = topol.find('\n', ff_pos) + 1

# Insert ligand.itp AFTER forcefield (ACPYPE ITP has atomtypes + moleculetype)
ligand_include = '\n; Include ligand topology (ACPYPE/GAFF2)\n#include "ligand.itp"\n'
new_topol = topol[:ff_end] + ligand_include + topol[ff_end:]

# Get ligand moleculetype name from ligand.itp
with open('ligand.itp', 'r') as f:
    itp_content = f.read()

moltype_match = re.search(r'\[ moleculetype \]\s*\n;.*\n\s*(\S+)', itp_content)
if moltype_match:
    lig_name = moltype_match.group(1)
else:
    lig_name = 'LIG'
print(f'Ligand moleculetype name: {lig_name}')

# Add ligand to [ molecules ] section
if lig_name not in new_topol.split('[ molecules ]')[-1]:
    new_topol += f'\n{lig_name}     1\n'

with open('topol.top', 'w') as f:
    f.write(new_topol)

print('Topology updated!')
os.chdir(WORK_DIR)

## Step 7: Solvate + Ions

In [ ]:
%%bash
set -e
source /usr/local/gromacs/bin/GMXRC
cd /kaggle/working/Luteolin_PDE5A/topol
gmx editconf -f complex.gro -o box.gro -c -d 1.2 -bt dodecahedron
gmx solvate -cp box.gro -cs spc216.gro -o solvated.gro -p topol.top
echo 'Solvated: OK'

In [ ]:
# Create ions.mdp
ions_mdp = '''integrator = steep
emtol = 1000.0
emstep = 0.01
nsteps = 50000
nstlist = 10
cutoff-scheme = Verlet
coulombtype = cutoff
rcoulomb = 1.0
rvdw = 1.0
pbc = xyz
'''
with open(TOPOL_DIR / 'ions.mdp', 'w') as f:
    f.write(ions_mdp)
print('ions.mdp created')

In [ ]:
%%bash
set -e
source /usr/local/gromacs/bin/GMXRC
cd /kaggle/working/Luteolin_PDE5A/topol
gmx grompp -f ions.mdp -c solvated.gro -p topol.top -o ions.tpr -maxwarn 2
printf 'SOL\n' | gmx genion -s ions.tpr -o system.gro -p topol.top -pname NA -nname CL -neutral -conc 0.15
echo 'Ions: OK'

## Step 8: Energy Minimization (GPU)

In [ ]:
em_mdp = '''integrator = steep
emtol = 1000.0
emstep = 0.01
nsteps = 50000
nstlist = 10
cutoff-scheme = Verlet
coulombtype = PME
rcoulomb = 1.0
rvdw = 1.0
pbc = xyz
'''
with open(WORK_DIR / 'em' / 'em.mdp', 'w') as f:
    f.write(em_mdp)
print('em.mdp created')

In [ ]:
%%bash
set -e
source /usr/local/gromacs/bin/GMXRC
cd /kaggle/working/Luteolin_PDE5A
gmx grompp -f em/em.mdp -c topol/system.gro -p topol/topol.top -o em/em.tpr -maxwarn 2
# GPU acceleration for energy minimization
gmx mdrun -v -deffnm em/em -nb gpu
echo '\n=== ENERGY MINIMIZATION COMPLETE ==='

## Step 9: NVT Equilibration (100 ps) - GPU

In [ ]:
nvt_steps = int(CONFIG['nvt_time_ps'] * 1000 / CONFIG['timestep_fs'])

nvt_mdp = f'''define = -DPOSRES
integrator = md
nsteps = {nvt_steps}
dt = {dt}
nstxout = 5000
nstvout = 5000
nstenergy = 5000
nstlog = 5000
continuation = no
constraint_algorithm = lincs
constraints = h-bonds
lincs_iter = 1
lincs_order = 4
cutoff-scheme = Verlet
nstlist = 10
rcoulomb = 1.0
rvdw = 1.0
coulombtype = PME
pme_order = 4
fourierspacing = 0.16
tcoupl = V-rescale
tc-grps = Protein Non-Protein
tau_t = 0.1 0.1
ref_t = {CONFIG['temperature_k']} {CONFIG['temperature_k']}
pcoupl = no
pbc = xyz
DispCorr = EnerPres
gen_vel = yes
gen_temp = {CONFIG['temperature_k']}
gen_seed = -1
'''
with open(WORK_DIR / 'nvt' / 'nvt.mdp', 'w') as f:
    f.write(nvt_mdp)
print(f'NVT: {CONFIG["nvt_time_ps"]} ps, {nvt_steps} steps')

In [ ]:
%%bash
set -e
source /usr/local/gromacs/bin/GMXRC
cd /kaggle/working/Luteolin_PDE5A
gmx grompp -f nvt/nvt.mdp -c em/em.gro -r em/em.gro -p topol/topol.top -o nvt/nvt.tpr -maxwarn 2
# Full GPU acceleration: non-bonded, PME, bonded
gmx mdrun -v -deffnm nvt/nvt -nb gpu -pme gpu -bonded gpu
echo '\n=== NVT EQUILIBRATION COMPLETE ==='

## Step 10: NPT Equilibration (100 ps) - GPU

In [ ]:
npt_steps = int(CONFIG['npt_time_ps'] * 1000 / CONFIG['timestep_fs'])

npt_mdp = f'''define = -DPOSRES
integrator = md
nsteps = {npt_steps}
dt = {dt}
nstxout = 5000
nstvout = 5000
nstenergy = 5000
nstlog = 5000
continuation = yes
constraint_algorithm = lincs
constraints = h-bonds
lincs_iter = 1
lincs_order = 4
cutoff-scheme = Verlet
nstlist = 10
rcoulomb = 1.0
rvdw = 1.0
coulombtype = PME
pme_order = 4
fourierspacing = 0.16
tcoupl = V-rescale
tc-grps = Protein Non-Protein
tau_t = 0.1 0.1
ref_t = {CONFIG['temperature_k']} {CONFIG['temperature_k']}
pcoupl = Parrinello-Rahman
pcoupltype = isotropic
tau_p = 2.0
ref_p = 1.0
compressibility = 4.5e-5
refcoord_scaling = com
pbc = xyz
DispCorr = EnerPres
gen_vel = no
'''
with open(WORK_DIR / 'npt' / 'npt.mdp', 'w') as f:
    f.write(npt_mdp)
print(f'NPT: {CONFIG["npt_time_ps"]} ps, {npt_steps} steps')

In [ ]:
%%bash
set -e
source /usr/local/gromacs/bin/GMXRC
cd /kaggle/working/Luteolin_PDE5A
gmx grompp -f npt/npt.mdp -c nvt/nvt.gro -r nvt/nvt.gro -t nvt/nvt.cpt -p topol/topol.top -o npt/npt.tpr -maxwarn 2
# Full GPU acceleration
gmx mdrun -v -deffnm npt/npt -nb gpu -pme gpu -bonded gpu
echo '\n=== NPT EQUILIBRATION COMPLETE ==='

## Step 11: Production MD (50 ns with checkpoints) - GPU

In [ ]:
md_mdp = f'''integrator = md
nsteps = {NSTEPS_PER_SEGMENT}
dt = {dt}
nstxout = 0
nstvout = 0
nstxout-compressed = 5000
nstenergy = 5000
nstlog = 5000
continuation = yes
constraint_algorithm = lincs
constraints = h-bonds
lincs_iter = 1
lincs_order = 4
cutoff-scheme = Verlet
nstlist = 10
rcoulomb = 1.0
rvdw = 1.0
coulombtype = PME
pme_order = 4
fourierspacing = 0.16
tcoupl = V-rescale
tc-grps = Protein Non-Protein
tau_t = 0.1 0.1
ref_t = {CONFIG['temperature_k']} {CONFIG['temperature_k']}
pcoupl = Parrinello-Rahman
pcoupltype = isotropic
tau_p = 2.0
ref_p = 1.0
compressibility = 4.5e-5
pbc = xyz
DispCorr = EnerPres
gen_vel = no
'''
with open(WORK_DIR / 'md' / 'md.mdp', 'w') as f:
    f.write(md_mdp)
print(f'Production MD: {CONFIG["checkpoint_interval_ns"]} ns per segment')
print(f'Total: {CONFIG["total_time_ns"]} ns in {NUM_SEGMENTS} segments')

In [ ]:
import subprocess
import time
from datetime import datetime

# Source GMXRC for subprocess
GMX_ENV = os.environ.copy()
GMX_ENV['PATH'] = '/usr/local/gromacs/bin:' + GMX_ENV.get('PATH', '')
GMX_ENV['LD_LIBRARY_PATH'] = '/usr/local/gromacs/lib:' + GMX_ENV.get('LD_LIBRARY_PATH', '')

def run_md_segment(segment_num, resume=False):
    start_ns = segment_num * CONFIG['checkpoint_interval_ns']
    end_ns = (segment_num + 1) * CONFIG['checkpoint_interval_ns']
    
    print(f'\n{"="*60}')
    print(f'Segment {segment_num + 1}/{NUM_SEGMENTS}: {start_ns}-{end_ns} ns')
    print(f'Started: {datetime.now().strftime("%H:%M:%S")}')
    print(f'{"="*60}')
    
    os.chdir(WORK_DIR)
    
    if segment_num == 0 and not resume:
        # First segment - prepare from NPT
        result = subprocess.run([
            'gmx', 'grompp', '-f', 'md/md.mdp', '-c', 'npt/npt.gro',
            '-t', 'npt/npt.cpt', '-p', 'topol/topol.top', '-o', 'md/md.tpr', '-maxwarn', '2'
        ], capture_output=True, text=True, env=GMX_ENV)
        if result.returncode != 0:
            print(f'grompp error: {result.stderr}')
            return False
        # GPU accelerated production MD
        subprocess.run(['gmx', 'mdrun', '-deffnm', 'md/md', '-v', '-nb', 'gpu', '-pme', 'gpu', '-bonded', 'gpu'], env=GMX_ENV)
    else:
        # CRITICAL FIX: Extend TPR time limit before continuing!
        # Without this, GROMACS sees target already reached and exits immediately
        extend_time_ps = CONFIG['checkpoint_interval_ns'] * 1000  # ns to ps
        print(f'Extending simulation time by {extend_time_ps} ps...')
        subprocess.run([
            'gmx', 'convert-tpr', '-s', 'md/md.tpr',
            '-extend', str(extend_time_ps), '-o', 'md/md.tpr'
        ], check=True, env=GMX_ENV)
        
        # Continue from checkpoint with GPU (use -noappend for safer file handling)
        subprocess.run(['gmx', 'mdrun', '-deffnm', 'md/md', '-cpi', 'md/md.cpt', '-noappend', '-v', '-nb', 'gpu', '-pme', 'gpu', '-bonded', 'gpu'], env=GMX_ENV)
    
    # Save checkpoint
    checkpoint_name = f'checkpoint_{end_ns}ns'
    checkpoint_dir = WORK_DIR / 'checkpoints' / checkpoint_name
    checkpoint_dir.mkdir(exist_ok=True)
    
    # Copy all MD output files (handles -noappend part files like md.part0002.xtc)
    for f in (WORK_DIR / 'md').glob('md*'):
        if f.suffix in ['.xtc', '.edr', '.log', '.cpt', '.gro', '.tpr']:
            shutil.copy(f, checkpoint_dir / f.name)
    
    # Copy to output
    output_checkpoint = OUTPUT_DIR / f"{CONFIG['complex_name']}_{checkpoint_name}"
    shutil.copytree(checkpoint_dir, output_checkpoint, dirs_exist_ok=True)
    
    print(f'\nCheckpoint saved: {checkpoint_name}')
    return True

In [ ]:
# RUN PRODUCTION MD
total_start = time.time()

for segment in range(NUM_SEGMENTS):
    segment_start = time.time()
    success = run_md_segment(segment, resume=(segment > 0))
    if not success:
        print(f'ERROR at segment {segment + 1}')
        break
    print(f'Segment time: {(time.time() - segment_start)/60:.1f} min')

print(f'\n{"="*60}')
print(f'PRODUCTION MD COMPLETE!')
print(f'Total time: {(time.time() - total_start)/3600:.2f} hours')
print(f'{"="*60}')

## Step 12: Analysis (Full 50 ns)

Concatenate all checkpoint trajectories, then analyze the complete 50 ns trajectory.

In [ ]:
# Step 12a: Concatenate all trajectory parts
os.chdir(WORK_DIR)

# Find files with correct order: md.xtc (Part 1) first, then md.part*.xtc
part_files = sorted((WORK_DIR / 'md').glob('md.part*.xtc'))
main_file = WORK_DIR / 'md' / 'md.xtc'

xtc_files = []
if main_file.exists():
    xtc_files.append(main_file)
xtc_files.extend(part_files)

print(f'Found {len(xtc_files)} trajectory files (in order):')
for f in xtc_files:
    print(f'  - {f.name} ({f.stat().st_size/1e6:.1f} MB)')

if len(xtc_files) > 1:
    xtc_list = ' '.join(str(f) for f in xtc_files)
    cmd = f'gmx trjcat -f {xtc_list} -o md/full_trajectory.xtc -cat'
    subprocess.run(cmd, shell=True, env=GMX_ENV)
    traj_file = 'md/full_trajectory.xtc'
else:
    traj_file = str(xtc_files[0]) if xtc_files else 'md/md.xtc'

print(f'\nUsing trajectory: {traj_file}')

In [ ]:
# Step 12b: PBC correction
cmd = f'echo "1\n0" | gmx trjconv -s md/md.tpr -f {traj_file} -o analysis/trajectory_clean.xtc -pbc mol -center -ur compact'
subprocess.run(cmd, shell=True, env=GMX_ENV)
clean_traj = 'analysis/trajectory_clean.xtc'
print('PBC correction: Done')

In [ ]:
%%bash
source /usr/local/gromacs/bin/GMXRC
cd /kaggle/working/Luteolin_PDE5A

echo '=== RMSD (Backbone) ==='
printf '4\n4\n' | gmx rms -s md/md.tpr -f analysis/trajectory_clean.xtc -o analysis/rmsd.xvg -tu ns

echo '=== RMSF (C-alpha) ==='
printf '3\n' | gmx rmsf -s md/md.tpr -f analysis/trajectory_clean.xtc -o analysis/rmsf.xvg -res

echo '=== Radius of Gyration ==='
printf '1\n' | gmx gyrate -s md/md.tpr -f analysis/trajectory_clean.xtc -o analysis/gyrate.xvg

echo 'Analysis complete!'
ls -la analysis/*.xvg

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def parse_xvg(filename):
    data = []
    with open(filename, 'r') as f:
        for line in f:
            if not line.startswith(('#', '@')):
                vals = [float(x) for x in line.split()]
                if vals: data.append(vals)
    return np.array(data)

rmsd = parse_xvg('analysis/rmsd.xvg')
rmsf = parse_xvg('analysis/rmsf.xvg')
rg = parse_xvg('analysis/gyrate.xvg')

print(f'Trajectory: {rmsd[0,0]:.1f} - {rmsd[-1,0]:.1f} ns')
print(f'RMSD: {rmsd[:,1].mean():.3f} ± {rmsd[:,1].std():.3f} nm')
print(f'Rg: {rg[:,1].mean():.3f} ± {rg[:,1].std():.3f} nm')

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
fig.suptitle(f"{CONFIG['complex_name']} - {CONFIG['total_time_ns']} ns MD", fontsize=14, fontweight='bold')

# RMSD
axes[0].plot(rmsd[:,0], rmsd[:,1], 'b-', lw=0.5)
axes[0].set(xlabel='Time (ns)', ylabel='RMSD (nm)', xlim=(0, rmsd[-1,0]))
axes[0].set_title(f'RMSD: {rmsd[:,1].mean():.3f}±{rmsd[:,1].std():.3f} nm')
axes[0].grid(alpha=0.3)

# RMSF
axes[1].plot(rmsf[:,0], rmsf[:,1], 'b-', lw=0.8)
axes[1].set(xlabel='Residue', ylabel='RMSF (nm)')
axes[1].set_title(f'RMSF: {rmsf[:,1].mean():.3f} nm avg')
axes[1].grid(alpha=0.3)

# Rg
t_ns = rg[:,0]/1000
axes[2].plot(t_ns, rg[:,1], 'orange', lw=0.5)
axes[2].set(xlabel='Time (ns)', ylabel='Rg (nm)', xlim=(0, t_ns.max()))
axes[2].set_title(f'Rg: {rg[:,1].mean():.3f}±{rg[:,1].std():.3f} nm')
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('analysis/md_analysis.png', dpi=300)
plt.show()

# Copy to output
shutil.copy('analysis/md_analysis.png', OUTPUT_DIR)
for xvg in ['rmsd.xvg', 'rmsf.xvg', 'gyrate.xvg']:
    shutil.copy(f'analysis/{xvg}', OUTPUT_DIR)
print(f'Results saved to {OUTPUT_DIR}')